# Generate CAM images for all test data. 
Before running this code copy the training files of the best/selected model (namded as "sgkf05-yolo11s") to 'classify-bestmodel' folder under runs file. If this file doesn't exist create one.

In [ ]:
# IMPORT PACKAGES
import os
import numpy as np
import pandas as pd
from ultralytics import YOLO
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import yaml
from tqdm import tqdm
from pytorch_grad_cam import (
    GradCAM, FEM, HiResCAM, ScoreCAM, GradCAMPlusPlus,
    AblationCAM, XGradCAM, EigenCAM, EigenGradCAM,
    LayerCAM, FullGrad, GradCAMElementWise, KPCA_CAM, ShapleyCAM,
    FinerCAM
)
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, preprocess_image
import torch
import cv2


In [ ]:
def load_images_from_folder(folder):
    return [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(('.jpg', '.png'))]

In [ ]:
def load_yaml_config(yaml_path, keys_to_extract):
    if not os.path.exists(yaml_path):
        return {key: None for key in keys_to_extract}
    with open(yaml_path, 'r') as f:
        config = yaml.safe_load(f)
    return {key: config.get(key, None) for key in keys_to_extract}

In [ ]:
# CONFIGURATION
classes = ['NRM', 'PSS']

data_dir = os.path.join('data', '2-splits')
seeds = [f'seed{r:02}' for r in range(1, 6)]
fold_counts = {'sgkf05': 5}

base_run_dir = os.path.join('runs', 'classify-bestmodel')
sgkf='sgkf05'
model_name = 'yolo11s'
model_base = f'{sgkf}-{model_name}'

# YAML fields to log from training
yaml_keys = [
    'epochs', 'batch', 'imgsz', 'optimizer', 'dropout', 'lr0',
    'weight_decay', 'model', 'pretrained', 'single_cls',
    'auto_augment', 'data'
]

In [ ]:
# Model interpretation
results = []

for seed in seeds:
    fold_range = range(1, fold_counts[sgkf] + 1)

    for fold in fold_range:

        fold_name = f'fold{fold:02}'
        train_id = f'train-{seed}-{fold_name}'
        test_id = f'test-{seed}-{fold_name}'

        data_test_dir = os.path.join(data_dir, sgkf, seed, fold_name, 'test')

        # Paths
        model_path = os.path.join(base_run_dir, model_base, train_id, 'weights', 'best.pt')
        yaml_path  = os.path.join(base_run_dir, model_base, train_id, 'args.yaml')
        data_test_dir   = os.path.join(data_dir, sgkf, seed, fold_name, 'test')

        # Skip if any key file is missing
        if not os.path.exists(model_path) or not os.path.exists(data_test_dir) or not os.path.exists(yaml_path):
            print(f"⚠️ Skipping (missing): {model_path} or {data_test_dir} or {yaml_path}")
            continue

        print(f"\n📌 Evaluating: {model_base} | {seed} | {fold_name}")
        model = YOLO(model_path)
        true_labels = []
        predicted_labels = []

        # Grad-CAM setup        
        gradcam_model = 'EigenGradCAM'      # 'GradCAM', 'GradCAMPlusPlus', 'EigenGradCAM', 'LayerCAM', 'HiResCAM', 'FinerCAM'
        
        gradcam_dir = os.path.join(base_run_dir, model_base, test_id, f'{gradcam_model}')
        os.makedirs(gradcam_dir, exist_ok=True)
        gradcam_image_dir = os.path.join(gradcam_dir, 'images')
        os.makedirs(gradcam_image_dir, exist_ok=True)
        gradcam_combined_dir = os.path.join(gradcam_dir, 'combined')        
        os.makedirs(gradcam_combined_dir, exist_ok=True)
    
        image_logs = []

        for cls_index, cls in enumerate(classes):
            cls_dir = os.path.normpath(os.path.join(data_test_dir, cls))
            images = load_images_from_folder(cls_dir)
            
            for img_path in tqdm(images, desc=f'{cls} images'):

                results_pred = model(img_path)
                predicted_class = results_pred[0].probs.top1
                true_labels.append(cls_index)

                # Log per-image inference
                image_logs.append({
                    'image_path': img_path,
                    'true_label': cls,
                    'predicted_label': classes[predicted_class],
                    'correct': int(cls_index == predicted_class)
                })

                predicted_labels.append(predicted_class)

                # Get inner PyTorch model
                cam_model = model.model
                # Set the correct last convolutional layer as target 
                target_layers = [cam_model.model[10].conv.conv]
                
                # Read and preprocess image
                img = cv2.imread(img_path)
                img = cv2.resize(img, (224, 224))
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_float = img_rgb.astype(np.float32) / 255.0

                device = next(cam_model.parameters()).device
                
                # Create tensor                      
                input_tensor = preprocess_image(img_rgb, mean=[0, 0, 0], std=[1, 1, 1]).to(device)
                input_tensor.requires_grad_(True)
                
                logits = cam_model(input_tensor)[0]
                pred_class = logits.argmax().item()

                # Apply GradCAM
                targets = [ClassifierOutputTarget(pred_class)]      
                #print(targets)
                
                # Create GradCAM object
                if gradcam_model == 'GradCAM':
                    cam = GradCAM(model=cam_model, target_layers=target_layers)
                elif gradcam_model == 'GradCAMPlusPlus':
                    cam = GradCAMPlusPlus(model=cam_model, target_layers=target_layers)
                elif gradcam_model == 'EigenGradCAM':
                    cam = EigenGradCAM(model=cam_model, target_layers=target_layers)                          
                elif gradcam_model == 'LayerCAM':
                    cam = LayerCAM(model=cam_model, target_layers=target_layers)
                elif gradcam_model == 'HiResCAM':
                    cam = HiResCAM(model=cam_model, target_layers=target_layers)
                elif gradcam_model == 'FinerCAM':
                    cam = FinerCAM(model=cam_model, target_layers=target_layers)

                grayscale_cam = cam(input_tensor=input_tensor, targets=targets)[0, :]      
                    
                cam_overlay = show_cam_on_image(img_float, grayscale_cam, use_rgb=True)

                base_filename = os.path.splitext(os.path.basename(img_path))[0]

                # Save Grad-CAM image
                gradcam_image_path = os.path.join(gradcam_image_dir, f'{base_filename}_{classes[pred_class]}_gradcam.jpg')
                cv2.imwrite(gradcam_image_path, cv2.cvtColor(cam_overlay, cv2.COLOR_RGB2BGR))

                # Save combined side-by-side figure (original | grad-cam)
                fig, axes = plt.subplots(1, 2, figsize=(6, 3), dpi=300)
                
                # Define font style
                font_settings = {
                    'family': 'Times New Roman',
                    'size': 10
                }
                
                # Left: Original
                axes[0].imshow(img_rgb)
                axes[0].set_title("Original", fontdict=font_settings, pad=10)
                axes[0].axis("off")

                # Right: Grad-CAM
                axes[1].imshow(cam_overlay)
                axes[1].set_title(f"{gradcam_model}", fontdict=font_settings, pad=10)
                axes[1].axis("off")

                # Save
                predicted_cls_name = classes[predicted_class]
                combined_fig_path = os.path.join(gradcam_combined_dir, f'{base_filename}_{predicted_cls_name}_combined.jpg')
                plt.tight_layout()
                plt.savefig(combined_fig_path, bbox_inches='tight', dpi=300)
                plt.close(fig)    
        


        # Save per-image predictions
        image_log_df = pd.DataFrame(image_logs)
        image_log_path = os.path.join(base_run_dir, model_base, test_id, 'image_predictions.csv')
        image_log_df.to_csv(image_log_path, index=False)
        print(f"📝 Per-image log saved to: {image_log_path}")

        # Confusion matrix
        cm = confusion_matrix(true_labels, predicted_labels, labels=[0, 1])

        # Create subfolder for confusion matrix output
        cm_output_dir = os.path.join(base_run_dir, model_base, test_id)
        os.makedirs(cm_output_dir, exist_ok=True)
        cm_fig_path = os.path.join(cm_output_dir, 'confusion_matrix.png')

        fig, ax = plt.subplots()
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
        disp.plot(cmap='Blues', ax=ax)
        plt.title(f"Confusion Matrix-{model_base}-{seed}-{fold_name}")
        plt.savefig(cm_fig_path)
        plt.close(fig)
        print(f"✅ Confusion matrix saved to: {cm_fig_path}")

        TP, TN, FP, FN = cm[1, 1], cm[0, 0], cm[0, 1], cm[1, 0]

        # Metrics
        accuracy = round((TP + TN) / np.sum(cm), 4)
        precision = round(TP / (TP + FP), 4) if (TP + FP) else 0.0
        sensitivity = round(TP / (TP + FN), 4) if (TP + FN) else 0.0
        specificity = round(TN / (TN + FP), 4) if (TN + FP) else 0.0
        f1_score = round(2 * (precision * sensitivity) / (precision + sensitivity), 4) if (precision + sensitivity) else 0.0

        # Load training configuration from YAML
        config_data = load_yaml_config(yaml_path, yaml_keys)
        expected_data_path = os.path.join(data_dir, sgkf, seed, fold_name)
        yaml_data_path = config_data.pop('data', '')
        config_data['data_path_match'] = expected_data_path in yaml_data_path

        # Append full record
        results.append({
            'sgkf': sgkf,
            'yolo_version': model_name,
            'seed': seed,
            'fold': fold_name,
            'TP': TP,
            'TN': TN,
            'FP': FP,
            'FN': FN,
            'accuracy': accuracy,
            'precision': precision,
            'sensitivity': sensitivity,
            'specificity': specificity,
            'f1_score': f1_score,
            **config_data
        })
# Save CSV with descriptive name
csv_name = f"test-{sgkf}-{model_name}.csv"
output_path = os.path.join(base_run_dir, model_base, csv_name)
df = pd.DataFrame(results)
drop_columns = ['auto_augment']
df.drop(columns=[col for col in drop_columns if col in df.columns], inplace=True)
df.to_csv(output_path, index=False)
print(f"\n📆 Output saved to: {output_path}")
